# Data Immersion Task 6.7 - Final Analysis Preparation

### Table of Contents

### 1. Realtor.com For Sale data
    1a. Consistency Checks
    1b. Data cleaning
    1c. Merging county information (for visualizing)
    
### 2. Realtor.com Rental data
    2a. Consistency Checks
    2b. Data cleaning
    2c. Merging county information (for visualizing)

In [1]:
# Importing libraries

import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
import scipy
import time
import datetime

In [2]:
# Defining path
path=r'D:\Adam\Employment\Data Analysis Course\Final Data Project'

## 1. Realtor.com For Sale Data

In [3]:
# Importing scraped data
df_rs = pd.read_csv(os.path.join(path, 'Data', 'Scraped', f'TysonZips_forsale_2023-06-16.csv'), dtype={'TysonZip': str}, index_col = False)

### 1a. Consistency Checks

In [4]:
# checking dataset info

df_rs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9959 entries, 0 to 9958
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   TysonZip      9959 non-null   object 
 1   Address1      9939 non-null   object 
 2   Address2      9939 non-null   object 
 3   Availability  9959 non-null   object 
 4   Price         9959 non-null   object 
 5   Beds          6974 non-null   object 
 6   Baths         6927 non-null   object 
 7   Sqft          6728 non-null   float64
dtypes: float64(1), object(7)
memory usage: 622.6+ KB


In [5]:
# Looking for any other strange values in the Price column:

df_rs['Price'].value_counts()

150000     89
125000     76
250000     75
45000      73
275000     67
           ..
1890000     1
537990      1
577990      1
529990      1
482000      1
Name: Price, Length: 2758, dtype: int64

In [6]:
# Found a "Contact for Price" row and need to drop the whole row

df_rs.drop(df_rs[df_rs['Price'] == 'Contact For Price'].index, inplace=True)

In [7]:
df_rs['Price'] = df_rs['Price'].astype('float64')

In [8]:
# Checking for duplicates (and removing them if there are any) - then checking if the numbers change

df_rs.drop_duplicates(inplace=True)

df_rs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9829 entries, 0 to 9958
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   TysonZip      9829 non-null   object 
 1   Address1      9810 non-null   object 
 2   Address2      9810 non-null   object 
 3   Availability  9829 non-null   object 
 4   Price         9829 non-null   float64
 5   Beds          6941 non-null   object 
 6   Baths         6895 non-null   object 
 7   Sqft          6696 non-null   float64
dtypes: float64(2), object(6)
memory usage: 691.1+ KB


In [9]:
# Checking for missing values
missing_values = df_rs.isnull().sum()

# total rows in dataframe
total_rows = df_rs.shape[0]

# making it show me all the rows
pd.options.display.max_rows = len(df_rs.dtypes)

# calculating percentage of missing values
percent_missing = ((missing_values / total_rows) * 100).round(2)

# Making a dataframe to show missing values
missing_value_table = pd.concat([missing_values, percent_missing], axis=1)

# Naming columns for sorting
missing_value_table.columns = ['Missing Values', 'Percent Missing']

# sorting the view based on most missing values %
missing_value_table.sort_values('Percent Missing', ascending=True, inplace=True)

print(missing_value_table)

              Missing Values  Percent Missing
TysonZip                   0             0.00
Availability               0             0.00
Price                      0             0.00
Address1                  19             0.19
Address2                  19             0.19
Beds                    2888            29.38
Baths                   2934            29.85
Sqft                    3133            31.88


In [10]:
# Sqft has a lot of missing values, and in looking at it, it appears that all the addresses with missing sqft values are
# just lots for sale. Which I don't believe helps the purpose of this analysis. So I'll remove all of those with missing
# sqft values. I'm not worried much about missing beds and baths info

df_rs.dropna(subset=['Sqft'], inplace=True)

In [11]:
df_rs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6696 entries, 0 to 9957
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   TysonZip      6696 non-null   object 
 1   Address1      6689 non-null   object 
 2   Address2      6689 non-null   object 
 3   Availability  6696 non-null   object 
 4   Price         6696 non-null   float64
 5   Beds          6632 non-null   object 
 6   Baths         6600 non-null   object 
 7   Sqft          6696 non-null   float64
dtypes: float64(2), object(6)
memory usage: 470.8+ KB


In [12]:
# Dropping all rows with missing addresses

df_rs.dropna(subset=['Address1'], inplace=True)

In [13]:
# counting number of unique values in each column

print(df_rs.nunique())

TysonZip          95
Address1        6667
Address2         253
Availability       5
Price           2158
Beds              22
Baths             32
Sqft            2431
dtype: int64


### 1b. Data cleaning

In [14]:
# Splitting up the Address2 data to create City, State, and Zip columns

df_rs['City'] = df_rs['Address2'].str.split(',', expand=True)[0]

df_rs['State'] = df_rs['Address2'].str.split(', ', expand=True)[1]
df_rs['State'] = df_rs['State'].str.split(' ', expand=True)[0]

# Some listings don't include zip code, so this will only capture those that do
df_rs['Zip'] = df_rs['Address2'].str.split(', ', expand=True)[1]
df_rs['Zip'] = df_rs['Zip'].str.split(' ', expand=True)[1]

In [15]:
df_rs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6689 entries, 0 to 9957
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   TysonZip      6689 non-null   object 
 1   Address1      6689 non-null   object 
 2   Address2      6689 non-null   object 
 3   Availability  6689 non-null   object 
 4   Price         6689 non-null   float64
 5   Beds          6625 non-null   object 
 6   Baths         6593 non-null   object 
 7   Sqft          6689 non-null   float64
 8   City          6689 non-null   object 
 9   State         6689 non-null   object 
 10  Zip           6197 non-null   object 
dtypes: float64(2), object(9)
memory usage: 627.1+ KB


In [16]:
# counting number of unique values in each column

print(df_rs.nunique())

TysonZip          95
Address1        6667
Address2         253
Availability       5
                ... 
Sqft            2431
City             220
State             28
Zip              202
Length: 11, dtype: int64


In [17]:
# dropping houses with duplicate addresses and sqft listed

df_rs.drop_duplicates(subset=['Address1', 'Sqft'], inplace=True)

In [18]:
df_rs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6678 entries, 0 to 9957
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   TysonZip      6678 non-null   object 
 1   Address1      6678 non-null   object 
 2   Address2      6678 non-null   object 
 3   Availability  6678 non-null   object 
 4   Price         6678 non-null   float64
 5   Beds          6616 non-null   object 
 6   Baths         6583 non-null   object 
 7   Sqft          6678 non-null   float64
 8   City          6678 non-null   object 
 9   State         6678 non-null   object 
 10  Zip           6186 non-null   object 
dtypes: float64(2), object(9)
memory usage: 626.1+ KB


In [19]:
# Still counting unique values in each object column

df_rs['Address2'].value_counts()

Seguin, TX 78155             494
Broken Bow, OK 74728         348
Fayetteville, AR 72704       264
Cumming, GA 30040            234
                            ... 
Gadsden, AL 35906              1
Mesa, WA 99343                 1
Eltopia, WA 99330              1
Pleasant Valley, IA 52767      1
Name: Address2, Length: 253, dtype: int64

In [20]:
# Still counting unique values in each object column

df_rs['Availability'].value_counts()

For Sale       4343
Pending        1872
Contingent      439
Foreclosure      19
Coming Soon       5
Name: Availability, dtype: int64

In [21]:
# Still counting unique values in each object column

# making it show me all the rows
pd.options.display.max_rows = len(df_rs['Beds'])

df_rs['Beds'].value_counts(dropna=False)

3         3238
4         1753
2          870
5          456
1          123
6          104
NaN         62
7           25
Studio      17
8           11
10           4
9            3
12           2
16           1
155          1
17           1
20           1
41           1
24           1
11           1
14           1
26           1
13           1
Name: Beds, dtype: int64

In [22]:
# Converting all "Studio" listings to "1", as we'll assume it's roughly equivalent

df_rs['Beds'] = df_rs['Beds'].replace('Studio', '1')

In [23]:
# converting Beds to float64 (I had to use float so I could conserve the NaN values)

df_rs['Beds'] = df_rs['Beds'].astype('float64')

In [24]:
# Dropping rows with too many beds (i.e. apartment complexes or the like)

df_rs.drop(df_rs[df_rs['Beds'] > 14].index, inplace=True)

In [25]:
df_rs['Beds'].value_counts(dropna=False)

3.0     3238
4.0     1753
2.0      870
5.0      456
1.0      140
6.0      104
NaN       62
7.0       25
8.0       11
10.0       4
9.0        3
12.0       2
11.0       1
14.0       1
13.0       1
Name: Beds, dtype: int64

In [26]:
# Still counting unique values in each object column

df_rs['Baths'].value_counts()

2        2562
2.5      1144
1         945
3         787
3.5       407
1.5       289
4         156
4.5       103
5          35
5.5        31
3.5+       27
2.5+       25
4.5+       18
6.5        10
5.5+        8
6           8
1.5+        6
8           5
6.5+        2
7           2
7.5+        1
0.5         1
7.5         1
0.5+        1
15          1
12.5+       1
10.5+       1
9.5+        1
Name: Baths, dtype: int64

In [27]:
# Replacing "+" values with normal values

df_rs['Baths'] = df_rs['Baths'].replace({'0.5+': '0.5', '1.5+': '1.5', '2.5+': '2.5', '3.5+': '3.5', '4.5+': '4.5', '5.5+': '5.5', '6.5+': '6.5', '7.5+': '7.5', '9.5+': '9.5', '10.5+': '10.5', '12.5+': '12.5'})

In [28]:
print(df_rs[df_rs['Baths'] == '15'])

     TysonZip                                     Address1  \
2711    72830  342 County Road 3446, Clarksville, AR 72830   

                   Address2 Availability    Price  Beds Baths    Sqft  \
2711  Clarksville, AR 72830      Pending  27000.0   3.0    15  1216.0   

             City State    Zip  
2711  Clarksville    AR  72830  


In [29]:
print(df_rs[df_rs['Baths'] == "15"])

     TysonZip                                     Address1  \
2711    72830  342 County Road 3446, Clarksville, AR 72830   

                   Address2 Availability    Price  Beds Baths    Sqft  \
2711  Clarksville, AR 72830      Pending  27000.0   3.0    15  1216.0   

             City State    Zip  
2711  Clarksville    AR  72830  


In [30]:
# I checked the address for the one with 15 beds and it was clearly a mistake. So changing it here.

df_rs.loc[df_rs['Baths'] == '15', 'Baths'] = '1.5'

In [31]:
# converting Baths to float64 (I had to use float so I could conserve the NaN values)

df_rs['Baths'] = df_rs['Baths'].astype('float64')

In [32]:
# Still counting unique values in each object column

df_rs['Sqft'].value_counts()

1800.0      33
1200.0      32
1600.0      25
1500.0      23
2000.0      23
1400.0      22
1300.0      22
960.0       22
2100.0      21
2400.0      21
1680.0      21
2200.0      20
1248.0      20
1440.0      19
1456.0      19
1000.0      18
2511.0      18
1983.0      18
1688.0      17
1636.0      17
1700.0      17
2368.0      17
1344.0      17
1266.0      17
1056.0      17
1152.0      16
1280.0      16
1296.0      15
1292.0      15
1575.0      15
1008.0      15
1092.0      15
1568.0      14
1567.0      14
1298.0      14
3000.0      14
1450.0      14
1862.0      14
1560.0      14
2031.0      14
1618.0      13
1796.0      13
1100.0      13
2300.0      13
1350.0      13
1667.0      13
1216.0      13
1626.0      12
1040.0      12
896.0       12
1512.0      12
1217.0      12
1616.0      12
1728.0      12
900.0       12
1910.0      12
1104.0      12
1120.0      12
864.0       11
1608.0      11
1920.0      11
1659.0      11
936.0       11
1792.0      11
1459.0      11
1080.0      11
1744.0    

In [33]:
print(df_rs[df_rs['Sqft'] > 15000])

     TysonZip                                    Address1  \
3861    75237       5619 Walnut Hill Ln, Dallas, TX 75229   
5762    27332         103 Grinnel Loop, Sanford, NC 27332   
5800    27332         106 Grinnel Loop, Sanford, NC 27332   
8529    75090          812 E Odneal St, Sherman, TX 75090   
8650    75090    820 N Sam Rayburn Fwy, Sherman, TX 75090   
9110    72704  4199 W Sante Fe St, Fayetteville, AR 72704   
9410    31092         202 St Charles Pl, Veinna, GA 31092   

                    Address2 Availability       Price  Beds  Baths      Sqft  \
3861        Dallas, TX 75229     For Sale  60000000.0  10.0   12.5   27092.0   
5762       Sanford, NC 27332     For Sale    390000.0   4.0    2.5   22137.0   
5800       Sanford, NC 27332     For Sale    390000.0   4.0    2.5   22137.0   
8529       Sherman, TX 75090     For Sale    590000.0   3.0    1.5   24000.0   
8650       Sherman, TX 75090      Pending   8000000.0   NaN    NaN   65986.0   
9110  Fayetteville, AR 72704   

Two of these are errors and the last three are multi-family units, so removing those and fixing the errors

In [34]:
df_rs.drop(df_rs[df_rs['Sqft'] > 30000].index, inplace=True)

In [35]:
df_rs.loc[df_rs['Sqft'] == 22137, 'Sqft'] = 2213

In [36]:
# Still counting unique values in each object column

df_rs['City'].value_counts(dropna=False)

Seguin                  669
Broken Bow              348
Cumming                 270
Fayetteville            264
Bowling Green           237
Sherman                 184
Sanford                 181
Shelbyville             173
Monroe                  173
Rogers                  158
Alexandria              153
North Richland Hills    147
Traverse City           140
Springdale              129
Rome                    123
Sedalia                 105
Goodlettsville          104
Vineland                101
Jacksonville             88
Chicago                  80
Council Bluffs           74
Hutchinson               70
Carthage                 70
Zeeland                  69
Clarksville              67
Russellville             67
Easley                   66
Caseyville               66
Hanceville               64
Haltom City              63
Jackson                  59
Vicksburg                58
Houston                  58
Enid                     58
Amherst                  56
Eufaula             

In [37]:
# Still counting unique values in each object column

df_rs['State'].value_counts(dropna=False)

TX    1334
AR    1077
KY     448
GA     437
OK     411
NC     382
TN     378
IA     295
MO     268
MI     259
AL     202
IL     180
KS     156
NE     154
MS     131
NJ     101
FL      88
IN      80
OH      72
SC      66
WA      34
ME      30
CA      28
WI      22
PA      13
VA       9
OR       7
SD       6
Name: State, dtype: int64

In [38]:
# Still counting unique values in each object column

df_rs['Zip'].value_counts(dropna=False)

78155    496
None     492
74728    352
72704    263
30040    234
42101    233
27332    171
72756    162
37160    160
28112    145
49684    140
75090    131
72764    129
76180    129
30161    117
65301    105
37072    104
08360    101
32254     88
60641     80
51501     74
67501     70
72830     68
62232     67
72802     67
41001     65
35077     62
77029     61
71602     59
39204     58
39183     58
73701     58
76117     57
36027     56
75633     55
49464     54
50703     52
65708     52
68137     51
29640     50
64507     50
48091     50
44001     49
63841     48
46947     47
72114     46
66106     45
38261     43
76384     43
79108     41
38343     40
71801     40
72616     39
72901     39
72834     36
35906     35
72958     35
47112     33
72638     32
04102     30
35950     30
71852     29
68850     29
50220     29
50588     28
94580     28
66801     27
42420     25
28610     24
51442     23
31092     22
54961     22
68462     22
75935     22
31730     21
38059     18
68305     18

In [39]:
print(df_rs['Zip'].nunique())

202


In [40]:
# Identifying null values in Zip

# Set the display options to show all rows
pd.set_option('display.max_rows', None)

print(df_rs[df_rs['Zip'].isnull()])

     TysonZip                                           Address1  \
302     44001  Brazos Plan, Hampshire Farms CommunityAmherst, OH   
307     44001  Grand Bahama Plan, Eagle Ridge Landing Communi...   
310     44001  Daffodil II Plan, Hampshire Farms CommunityAmh...   
317     44001  Honeysuckle Plan, Hampshire Farms CommunityAmh...   
319     44001  Azalea Plan, Hampshire Farms CommunityAmherst, OH   
321     44001  Eden Cay Plan, Eagle Ridge Landing CommunityAm...   
323     44001  Aruba Bay Plan, Eagle Ridge Landing CommunityA...   
324     44001  Grand Cayman Plan, Eagle Ridge Landing Communi...   
380     72764  DENTON Plan, Hylton Place CommunitySPRINGDALE, AR   
382     72764  COLEMAN Plan, Hylton Place CommunitySPRINGDALE...   
385     72764    DEAN Plan, Hylton Place CommunitySPRINGDALE, AR   
389     72764  Lot 14 Custom Plan, Sonora CommunitySPRINGDALE...   
397     72764  FRISCO Plan, Hylton Place CommunitySPRINGDALE, AR   
400     72764  Cooper Bonus Plan, Sonora Communi

In [41]:
# All the null values in "Zip" are actually potential house plans (I checked the website), and they don't represent individual
# addresses.  So I'm going to drop all of them

df_rs = df_rs.dropna(subset=['Zip'])

In [42]:
df_rs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6176 entries, 0 to 9957
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   TysonZip      6176 non-null   object 
 1   Address1      6176 non-null   object 
 2   Address2      6176 non-null   object 
 3   Availability  6176 non-null   object 
 4   Price         6176 non-null   float64
 5   Beds          6117 non-null   float64
 6   Baths         6087 non-null   float64
 7   Sqft          6176 non-null   float64
 8   City          6176 non-null   object 
 9   State         6176 non-null   object 
 10  Zip           6176 non-null   object 
dtypes: float64(4), object(7)
memory usage: 579.0+ KB


### 1c. Merging county information (to prepare for visualizations)

In [43]:
# Loading another dataframe so I can import county FIPS codes and merge that on the zip code

# Simplemaps.com county & zip data
zip_cnty = pd.read_csv(os.path.join(path, 'Data', 'Original', 'GeoData', 'uszips-counties.csv'), dtype={'county_fips': str, 'zip' : str}, index_col = False)

In [44]:
# Adding the columns and values I want from the zip_cnty dataframe to my df_rs dataframe
df_rs = df_rs.merge(zip_cnty[['zip', 'lat', 'lng', 'county_fips', 'county_name']], 
                        left_on='Zip', right_on='zip', how='left')

# Remove the redundant 'zip' column from the merge result
df_rs.drop('zip', axis=1, inplace=True)

#showing results to confirm
df_rs.head()

,TysonZip,Address1,Address2,Availability,Price,Beds,Baths,Sqft,City,State,Zip,lat,lng,county_fips,county_name
0,73701,"526 E Cottonwood Ln, Enid, OK 73701","Enid, OK 73701",For Sale,290000.0,3.0,2.0,1582.0,Enid,OK,73701,36.42323,-97.78833,40047,Garfield
1,73701,"111 S 14th St, Enid, OK 73701","Enid, OK 73701",For Sale,129900.0,3.0,2.5,1508.0,Enid,OK,73701,36.42323,-97.78833,40047,Garfield
2,73701,"701 W Nagel Ave, Enid, OK 73701","Enid, OK 73701",For Sale,84900.0,3.0,1.0,1040.0,Enid,OK,73701,36.42323,-97.78833,40047,Garfield
3,73701,"425 N Candlewood St, Enid, OK 73701","Enid, OK 73701",For Sale,125000.0,3.0,2.0,1824.0,Enid,OK,73701,36.42323,-97.78833,40047,Garfield
4,73701,"2822 E Paradise Ln, Enid, OK 73701","Enid, OK 73701",Pending,142000.0,2.0,2.0,2414.0,Enid,OK,73701,36.42323,-97.78833,40047,Garfield


In [45]:
df_rs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6176 entries, 0 to 6175
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   TysonZip      6176 non-null   object 
 1   Address1      6176 non-null   object 
 2   Address2      6176 non-null   object 
 3   Availability  6176 non-null   object 
 4   Price         6176 non-null   float64
 5   Beds          6117 non-null   float64
 6   Baths         6087 non-null   float64
 7   Sqft          6176 non-null   float64
 8   City          6176 non-null   object 
 9   State         6176 non-null   object 
 10  Zip           6176 non-null   object 
 11  lat           6176 non-null   float64
 12  lng           6176 non-null   float64
 13  county_fips   6176 non-null   object 
 14  county_name   6176 non-null   object 
dtypes: float64(6), object(9)
memory usage: 772.0+ KB


In [46]:
# Converting columns with leading zeroes to strings so that I can keep those leading zeroes when exporting

df_rs[['TysonZip', 'Zip', 'county_fips']] = df_rs[['TysonZip', 'Zip', 'county_fips']].astype(str)

In [47]:
# Exporting cleaned scraped dataset

# get today's date in the format YYYY-MM-DD
today = datetime.datetime.today().strftime('%Y-%m-%d')

df_rs.to_csv(os.path.join(path, 'Data', 'Cleaned', f'Realtor_Scraped_ForSale_{today}.csv'), index=False)

## 2. Realtor.com Rental data

In [48]:
# Importing scraped data

df_rr = pd.read_csv(os.path.join(path, 'Data', 'Scraped', f'TysonZips_rentals_2023-06-16.csv'), dtype={'TysonZip': str}, index_col = False)

### 2a. Consistency Checks

In [49]:
# checking dataset info

df_rr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1778 entries, 0 to 1777
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   TysonZip  1778 non-null   object
 1   Address1  1775 non-null   object
 2   Address2  1775 non-null   object
 3   Style     1778 non-null   object
 4   Price     1778 non-null   object
 5   Beds      1774 non-null   object
 6   Baths     1766 non-null   object
 7   Sqft      1577 non-null   object
dtypes: object(8)
memory usage: 111.2+ KB


In [50]:
# making it show me all the rows
pd.options.display.max_rows = len(df_rr['Price'].value_counts())

df_rr['Price'].value_counts()

1550                 38
1200                 34
750                  32
1250                 28
1450                 28
2400                 27
2000                 25
1800                 24
825                  23
1395                 23
1050                 21
725                  21
1100                 21
900                  20
1000                 19
1600                 19
1400                 19
1950                 19
1895                 19
850                  19
1850                 18
1795                 17
1095                 17
1500                 17
1649                 17
650                  16
3000                 16
2500                 16
1300                 15
2200                 15
1150                 15
1995                 15
950                  15
1495                 14
1625                 14
800                  14
1595                 14
2895                 13
1425                 13
1750                 13
1650                 13
2100            

In [51]:
# Dropping rows with "Contact For Price" value

df_rr.drop(df_rr[df_rr['Price'] == 'Contact For Price'].index, inplace=True)

In [52]:
# making Price an float64 dtype

df_rr = df_rr.astype({'Price': 'float64'})

In [53]:
# making it show me all the rows
pd.options.display.max_rows = len(df_rr['Sqft'].value_counts())

df_rr['Sqft'].value_counts()

700            40
1100           33
900            26
2000           26
1200           21
1372           18
625            18
650            17
800            16
750            15
500            15
1147           15
1300           14
694            13
1146           13
1000           13
1350           12
1729           12
1159           12
950            12
726            11
400            10
850            10
1067           10
1050            9
960             9
1400            8
550             8
1261            8
1500            8
1680            8
977             8
1212            7
957             7
1082            7
1298            7
300             7
660             7
1440            7
600             6
980             6
855             6
1256            6
2002            6
680             6
780             6
1450            6
1600            6
896             6
2032            5
630             5
1056            5
1255            5
640             5
564             5
2054      

In [54]:
# Keeping the lower sqft value from those that show sqft ranges, since these were properties that only listed a single
# price, and they were most likely listing "starting from" prices for a range of unit sizes (but I can't guess upper price)

df_rr['Sqft'] = df_rr['Sqft'].str.split(' - ').str[0]

In [55]:
# Convert sqft to numeric while keeping null values as null

df_rr['Sqft'] = df_rr['Sqft'].astype('float64')

In [56]:
# Checking for missing values

missing_values = df_rr.isnull().sum()

# total rows in dataframe
total_rows = df_rr.shape[0]

# making it show me all the rows
pd.options.display.max_rows = len(df_rr.dtypes)

# calculating percentage of missing values
percent_missing = ((missing_values / total_rows) * 100).round(2)

# Making a dataframe to show missing values
missing_value_table = pd.concat([missing_values, percent_missing], axis=1)

# Naming columns for sorting
missing_value_table.columns = ['Missing Values', 'Percent Missing']

# sorting the view based on most missing values %
missing_value_table.sort_values('Percent Missing', ascending=True, inplace=True)

print(missing_value_table)

          Missing Values  Percent Missing
TysonZip               0             0.00
Style                  0             0.00
Price                  0             0.00
Address1               3             0.17
Address2               3             0.17
Beds                   3             0.17
Baths                 10             0.56
Sqft                 197            11.13


In [57]:
# counting number of unique values in each column

print(df_rr.nunique())

TysonZip      78
Address1    1473
Address2      97
Style          7
Price        508
Beds          14
Baths         14
Sqft         689
dtype: int64


### 2b. Data cleaning

In [58]:
# Checking value counts on each column to look for more cleaning to be done

df_rr['Style'].value_counts(dropna=False)

Apartment         859
House             649
Townhome          102
Other              79
Condo              41
Condo/Townhome     23
Duplex/Triplex     17
Name: Style, dtype: int64

In [59]:
# More value counts for each column

# making it show me all the rows
pd.set_option('display.max_rows', None)

df_rr['Beds'].value_counts(dropna=False)

3             590
2             541
1             345
4             170
Studio         61
5              30
Studio+        13
2 - 3           5
1 - 2           5
6               3
NaN             3
1 - 3           1
150             1
9               1
Studio - 2      1
Name: Beds, dtype: int64

In [60]:
# I'm going to keep the first value in those that say "2-3" and stuff like that (for the same reason as above, because
# if they only listed one price, they probably listed the starting price, adn thus what would apply to the smallest unit)

df_rr['Beds'] = df_rr['Beds'].str.split(' - ').str[0]

In [61]:
# Changing Studio+ to Studio
df_rr['Beds'] = df_rr['Beds'].replace({'Studio+': 'Studio'})

In [62]:
# Can't imagine a rental place with 150 bedrooms

print(df_rr[df_rr['Beds'] == '150'])

    TysonZip        Address1          Address2  Style  Price Beds  Baths  \
942    28112  903 Walters St  Monroe, NC 28112  House  599.0  150  60.5+   

      Sqft  
942  500.0  


The beds and baths value makes no sense, but the apt seems legitimate (I looked it up).  I'll impute values of 1 bed and 0.5 baths, as that seems what's available from the description

In [63]:
df_rr['Beds'] = df_rr['Beds'].replace({'150': '1'})
df_rr['Baths'] = df_rr['Baths'].replace({'60.5+': '0.5'})

In [64]:
# Changing value of "Studio" to 1 Bed so that Beds can be turned into numbers (float)

df_rr['Beds'] = df_rr['Beds'].replace({'Studio': '1'})

In [65]:
df_rr['Beds'].value_counts()

3    590
2    546
1    427
4    170
5     30
6      3
9      1
Name: Beds, dtype: int64

In [66]:
df_rr['Beds'] = df_rr['Beds'].astype('float64')

In [67]:
# More value counts for each column

df_rr['Baths'].value_counts(dropna=False)

1          772
2          694
2.5        129
3.5         44
3           43
1.5         32
4.5         17
4           12
NaN         10
1 - 2        7
0.5+         6
5            1
0.5          1
1 - 2.5      1
3.5+         1
Name: Baths, dtype: int64

In [68]:
# Keeping the first number when there's a -, for the same reason as above

df_rr['Baths'] = df_rr['Baths'].str.split(' - ').str[0]

In [69]:
# Changing Baths+ to value without +

df_rr['Baths'] = df_rr['Baths'].replace({'0.5+': '0.5', '1.5+': '1.5', '2.5+': '2.5', '3.5+': '3.5', '4.5+': '4.5', '5.5+': '5.5'})

In [70]:
# More value counts for each column

df_rr['Baths'].value_counts(dropna=False)

1      780
2      694
2.5    129
3.5     45
3       43
1.5     32
4.5     17
4       12
NaN     10
0.5      7
5        1
Name: Baths, dtype: int64

In [71]:
df_rr['Baths'] = df_rr['Baths'].astype('float64')

In [72]:
# Splitting Address 2 into city, state, and zip

df_rr['City'] = df_rr['Address2'].str.split(',', expand=True)[0]
# State abbreviation
df_rr['State'] = df_rr['Address2'].str.split(', ', expand=True)[1]
df_rr['State'] = df_rr['State'].str.split(' ', expand=True)[0]
# zip code
df_rr['Zip'] = df_rr['Address2'].str.split(', ', expand=True)[1]
df_rr['Zip'] = df_rr['Zip'].str.split(' ', expand=True)[1]

In [73]:
# counting number of unique values in each column

print(df_rr.nunique())

TysonZip      78
Address1    1473
Address2      97
Style          7
Price        508
Beds           7
Baths         10
Sqft         689
City          90
State         25
Zip           82
dtype: int64


This time, 4 more zip codes had rental properties (within 3 mile radius) after none were found in those 4 tyzon zip code areas

In [74]:
df_rr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1770 entries, 0 to 1777
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   TysonZip  1770 non-null   object 
 1   Address1  1767 non-null   object 
 2   Address2  1767 non-null   object 
 3   Style     1770 non-null   object 
 4   Price     1770 non-null   float64
 5   Beds      1767 non-null   float64
 6   Baths     1760 non-null   float64
 7   Sqft      1573 non-null   float64
 8   City      1767 non-null   object 
 9   State     1767 non-null   object 
 10  Zip       1767 non-null   object 
dtypes: float64(4), object(7)
memory usage: 165.9+ KB


In [75]:
# More value counts for each column

df_rr['Address2'].value_counts(dropna=False)

North Richland Hills, TX 76180    193
Chicago, IL 60641                 135
Fayetteville, AR 72704            133
Seguin, TX 78155                  125
Bowling Green, KY 42101           123
Sherman, TX 75090                  75
North Little Rock, AR 72114        69
Cumming, GA 30040                  67
Fort Smith, AR 72901               56
Jacksonville, FL 32254             49
Springdale, AR 72764               44
Sanford, NC 27332                  42
Haltom City, TX 76117              40
Waterloo, IA 50703                 37
Goodlettsville, TN 37072           30
Easley, SC 29640                   30
Dallas, TX 75237                   27
Rogers, AR 72756                   23
Omaha, NE 68137                    23
Jackson, MS 39204                  23
Warren, MI 48091                   23
Shelbyville, TN 37160              22
Houston, TX 77029                  20
Monroe, NC 28112                   19
Hutchinson, KS 67501               18
Emporia, KS 66801                  17
Kansas City,

In [76]:
# More value counts for each column

df_rr['City'].value_counts(dropna=False)

North Richland Hills    195
Chicago                 135
Fayetteville            134
Seguin                  125
Bowling Green           123
Sherman                  75
North Little Rock        69
Cumming                  67
Fort Smith               56
Jacksonville             49
Springdale               45
Sanford                  42
Haltom City              41
Waterloo                 37
Easley                   30
Goodlettsville           30
Dallas                   27
Omaha                    23
Warren                   23
Rogers                   23
Jackson                  23
Shelbyville              22
Houston                  20
Monroe                   19
Hutchinson               18
Kansas City              17
Emporia                  17
Council Bluffs           16
Cullman                  16
Rome                     15
Vineland                 15
Traverse City            13
Logansport               12
Amherst                  11
Enid                     10
Cincinnati          

In [77]:
# More value counts for each column

df_rr['State'].value_counts(dropna=False)

TX     500
AR     351
IL     148
KY     125
GA      82
NC      65
IA      59
TN      59
KS      54
FL      49
MI      39
NE      34
SC      30
MS      30
AL      27
OH      24
MO      20
NJ      15
IN      13
OK      11
CA      10
PA       9
WI       7
ME       4
NaN      3
VA       2
Name: State, dtype: int64

In [78]:
# More value counts for each column

df_rr['Zip'].value_counts(dropna=False)

76180    194
60641    135
72704    133
78155    127
42101    123
75090     76
72114     73
30040     67
72901     56
32254     49
72764     45
76117     43
27332     42
50703     37
37072     30
29640     30
75237     27
48091     24
68137     23
72756     23
39204     23
37160     22
77029     21
28112     19
67501     18
66801     17
66106     17
51501     16
30161     15
08360     15
35055     15
45246     13
49684     13
46947     12
44001     11
94580     10
73701     10
17557      9
54961      7
71602      7
39183      7
63841      7
35906      6
38261      6
62234      6
79108      5
64507      5
68507      5
72801      4
72802      4
50588      4
04102      4
28610      3
64020      3
35950      3
62208      3
65301      3
75633      3
68505      3
72715      3
NaN        3
76384      3
49464      2
72638      2
35031      2
61257      2
67505      2
24586      2
68776      2
65708      2
41001      2
50220      1
62269      1
28697      1
74728      1
62204      1
47112      1

In [79]:
# Identifying null values for city state and zip (must be null for address2)

print(df_rr[df_rr['Zip'].isnull()])

    TysonZip Address1 Address2      Style   Price  Beds  Baths    Sqft City  \
18     79108      NaN      NaN      House  2500.0   4.0    3.5  2902.0  NaN   
119    42101      NaN      NaN  Apartment   600.0   1.0    1.0     NaN  NaN   
229    60641      NaN      NaN      House  1575.0   2.0    1.0  1100.0  NaN   

    State  Zip  
18    NaN  NaN  
119   NaN  NaN  
229   NaN  NaN  


In [80]:
# Because the one above has no address, even though it has a price, but it's probably better to just drop it

df_rr.dropna(subset=['Address2'], inplace=True)

### 2c. Merging county information (to prepare for visualizations)

In [81]:
# Adding the columns and values I want from the zip_cnty dataframe to my df_rs dataframe
df_rr = df_rr.merge(zip_cnty[['zip', 'lat', 'lng', 'county_fips', 'county_name']], 
                        left_on='Zip', right_on='zip', how='left')

# Remove the redundant 'zip' column from the merge result
df_rr.drop('zip', axis=1, inplace=True)

#showing results to confirm
df_rr.head()

,TysonZip,Address1,Address2,Style,Price,Beds,Baths,Sqft,City,State,Zip,lat,lng,county_fips,county_name
0,73701,1201 E Broadway Ave Apt B,"Enid, OK 73701",House,600.0,1.0,1.0,956.0,Enid,OK,73701,36.42323,-97.78833,40047,Garfield
1,73701,1201 E Broadway Ave Apt J,"Enid, OK 73701",House,695.0,1.0,1.0,NaN,Enid,OK,73701,36.42323,-97.78833,40047,Garfield
2,73701,1910 N Grand St,"Enid, OK 73701",House,2000.0,2.0,1.0,NaN,Enid,OK,73701,36.42323,-97.78833,40047,Garfield
3,73701,724 E Park St,"Enid, OK 73701",House,750.0,4.0,1.0,1536.0,Enid,OK,73701,36.42323,-97.78833,40047,Garfield
4,73701,3670 Antelope Dr,"Enid, OK 73701",House,1050.0,3.0,1.5,1095.0,Enid,OK,73701,36.42323,-97.78833,40047,Garfield


In [82]:
df_rr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1767 entries, 0 to 1766
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   TysonZip     1767 non-null   object 
 1   Address1     1767 non-null   object 
 2   Address2     1767 non-null   object 
 3   Style        1767 non-null   object 
 4   Price        1767 non-null   float64
 5   Beds         1764 non-null   float64
 6   Baths        1757 non-null   float64
 7   Sqft         1571 non-null   float64
 8   City         1767 non-null   object 
 9   State        1767 non-null   object 
 10  Zip          1767 non-null   object 
 11  lat          1767 non-null   float64
 12  lng          1767 non-null   float64
 13  county_fips  1767 non-null   object 
 14  county_name  1767 non-null   object 
dtypes: float64(6), object(9)
memory usage: 220.9+ KB


In [83]:
# Converting columns with leading zeroes to strings so that I can keep those leading zeroes when exporting

df_rs[['TysonZip', 'Zip', 'county_fips']] = df_rs[['TysonZip', 'Zip', 'county_fips']].astype(str)

In [84]:
# Exporting cleaned scraped dataset

df_rr.to_csv(os.path.join(path, 'Data', 'Cleaned', f'Realtor_Scraped_Rental_{today}.csv'), index=False)